In [ ]:
import keras
import sys
import os
sys.path.insert(0, os.path.abspath(os.path.join(os.path.dirname('__file__'), '..')))
import mdn
import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt
%matplotlib inline

# # Only for GPU use:
# import os
# os.environ["CUDA_VISIBLE_DEVICES"]="1"

# import tensorflow as tf
# config = tf.ConfigProto()
# config.gpu_options.allow_growth = True
# sess = tf.Session(config=config)
# from keras import backend as K
# K.set_session(sess)

In [ ]:
# Load and process raw OSC saved data
data = pd.read_csv("../datasets/2018-06-21-block-perf1.txt")
data['dt'] = data.time.diff().fillna(0)
data['x'] = data.message.apply((lambda x: int(x.split(' ')[2]))) / 127.0
data['dx'] = data.x.diff().fillna(0)
data['y'] = data.message.apply((lambda x: int(x.split(' ')[3]))) / 127.0
data['dy'] = data.y.diff().fillna(0)
data['z'] = data.message.apply((lambda x: int(x.split(' ')[4]))) / 127.0
data['dz'] = data.z.diff().fillna(0)

# Save as numpy array
cleaned_data = data[['dx', 'dy', 'dz', 'dt']]
array_version = np.array(cleaned_data)
np.savez('roli-block-session-data-diff.npz', array_version)

# Have a look at some details:
print("Here's an excerpt of data:")
print(cleaned_data[100:110])
cleaned_data.describe()

In [ ]:
# Test loading
with np.load('roli-block-session-data-diff.npz') as data:
    dataset = data['arr_0']
print("Here's the data shape:")
print(dataset.shape)
print("Corpus data points between 100 and 120:")
print(dataset[100:120])

train_dataset = dataset[:10000]
val_dataset = dataset[10000:]

print("Some statistics about the dataset:")
pd.DataFrame(dataset).describe()

In [ ]:
# Training Hyperparameters:
SEQ_LEN = 30
BATCH_SIZE = 64
HIDDEN_UNITS = 128
LSTM_LAYERS = 1
EPOCHS = 100
SEED = 2345  # set random seed for reproducibility
random.seed(SEED)
np.random.seed(SEED)
OUTPUT_DIMENSION = 4
NUMBER_MIXTURES = 3

inputs = keras.layers.Input(shape=(SEQ_LEN,OUTPUT_DIMENSION), name='inputs')

lstm_inputs = inputs # input for first LSTM layer
for i in range(LSTM_LAYERS):
    lstm_out = keras.layers.LSTM(HIDDEN_UNITS, name='lstm'+str(i), return_sequences=True)(lstm_inputs)
    lstm_inputs = lstm_out
    
#lstm1_out = keras.layers.LSTM(HIDDEN_UNITS, name='lstm1', return_sequences=True)(inputs)
#lstm2_out = keras.layers.LSTM(HIDDEN_UNITS, name='lstm2', return_sequences=True)(lstm1_out)

mdn_out = keras.layers.TimeDistributed(mdn.MDN(OUTPUT_DIMENSION, NUMBER_MIXTURES, name='mdn_outputs'), name='td_mdn')(lstm_out)

model = keras.models.Model(inputs=inputs, outputs=mdn_out)
model.compile(loss=mdn.get_mixture_loss_func(OUTPUT_DIMENSION,NUMBER_MIXTURES), optimizer='adam')
model.summary()

In [ ]:
# Functions for slicing up data
def slice_sequence_examples(sequence, num_steps):
    xs = []
    for i in range(len(sequence) - num_steps - 1):
        example = sequence[i: i + num_steps]
        xs.append(example)
    return xs

def seq_to_overlapping_format(examples):
    """Takes sequences of seq_len+1 and returns overlapping
    sequences of seq_len."""
    xs = []
    ys = []
    for ex in examples:
        xs.append(ex[:-1])
        ys.append(ex[1:])
    return (xs,ys)

# Prepare training data as X and Y.
slices = []
#for seq in train_set:
slices =  slice_sequence_examples(train_dataset, SEQ_LEN+1)
X, y = seq_to_overlapping_format(slices)

X = np.array(X)
y = np.array(y)

print("Number of training examples:")
print("X:", X.shape)
print("y:", y.shape)

# Prepare validation data as X and Y.
slices = []
slices +=  slice_sequence_examples(val_dataset, SEQ_LEN+1)
Xval, yval = seq_to_overlapping_format(slices)

Xval = np.array(Xval)
yval = np.array(yval)

print("Number of validation examples:")
print("X:", Xval.shape)
print("y:", yval.shape)

In [ ]:
# Fit the model
filepath="lightpad_mdnrnn-{epoch:02d}.hdf5"
checkpoint = keras.callbacks.ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
callbacks = [keras.callbacks.TerminateOnNaN(), checkpoint]

history = model.fit(X, y, batch_size=BATCH_SIZE, epochs=EPOCHS, callbacks=callbacks, validation_data=(Xval,yval))
model.save('lightpad_mdnrnn_model_time_distributed.h5')  # creates a HDF5 file 'my_model.h5'

In [ ]:
hist = pd.DataFrame.from_dict(history.history)
hist.plot()

# Decoding Model

Let's try it out!


In [ ]:
# Decoding Model
# Same as training model except for dimension and mixtures.

# decoder.add(keras.layers.LSTM(HIDDEN_UNITS, batch_input_shape=(1,1,OUTPUT_DIMENSION), return_sequences=True, stateful=True))
# decoder.add(keras.layers.LSTM(HIDDEN_UNITS, stateful=True))
# decoder.add(mdn.MDN(OUTPUT_DIMENSION, NUMBER_MIXTURES))
# decoder.compile(loss=mdn.get_mixture_loss_func(OUTPUT_DIMENSION,NUMBER_MIXTURES), optimizer=keras.optimizers.Adam())
# decoder.summary()


inputs = keras.layers.Input(shape=(1,OUTPUT_DIMENSION), name='inputs')

lstm_inputs = inputs # input for first LSTM layer
for i in range(LSTM_LAYERS):
    lstm_out = keras.layers.LSTM(HIDDEN_UNITS, name='lstm'+str(i), return_sequences=True)(lstm_inputs)
    lstm_inputs = lstm_out
    
#last_output = Lambda(lambda x: x[:,-1,:], output_shape=(1,) + input_shape[2:])(lstm_out)

mdn_out = mdn.MDN(OUTPUT_DIMENSION, NUMBER_MIXTURES, name='mdn_outputs')(lstm_out)
decoder = keras.models.Model(inputs=inputs, outputs=mdn_out)
decoder.compile(loss=mdn.get_mixture_loss_func(OUTPUT_DIMENSION,NUMBER_MIXTURES), optimizer='adam')
decoder.summary()

decoder.load_weights('lightpad_mdnrnn_model_time_distributed.h5') # load weights independently from file

In [ ]:
# Performance functions
input_colour = 'darkblue'
gen_colour = 'firebrick'
plt.style.use('seaborn-talk')

def perf_df_to_array(perf_df, xylim=1.0, dtlim=5.0):
    """Converts a dataframe of a performance into array a,b,dt format."""
    perf_df['dt'] = perf_df.time.diff()
    perf_df.dt = perf_df.dt.fillna(0.0)
    perf_df.set_value(perf_df[perf_df.dt > 5].index, 'dt', dtlim)
    perf_df.set_value(perf_df[perf_df.dt < 0].index, 'dt', 0.0)
    perf_df.set_value(perf_df[perf_df.x > 1].index, 'x', xylim)
    perf_df.set_value(perf_df[perf_df.x < 0].index, 'x', 0.0)
    perf_df.set_value(perf_df[perf_df.y > 1].index, 'y', xylim)
    perf_df.set_value(perf_df[perf_df.y < 0].index, 'y', 0.0)
    return np.array(perf_df[['x', 'y', 'dt']])

def perf_array_to_df(perf_array):
    """Converts an array of a performance (a,b,dt format) into a dataframe."""
    perf_array = perf_array.T
    perf_df = pd.DataFrame({'dx': perf_array[0], 'dy': perf_array[1], 'dz': perf_array[2], 'dt': perf_array[3]})
    perf_df.dt = perf_df.dt.clip(lower=0.0)
    perf_df.dx = perf_df.dx.clip(lower=-1.0, upper=1.0)
    perf_df.dy = perf_df.dy.clip(lower=-1.0, upper=1.0)
    perf_df.dz = perf_df.dz.clip(lower=-1.0, upper=1.0)
    perf_df['time'] = perf_df.dt.cumsum()
    perf_df['x'] = perf_df.dx.cumsum()
    perf_df['y'] = perf_df.dy.cumsum()
    perf_df['z'] = perf_df.dz.cumsum()
    # As a rule of thumb, could classify taps with dt>0.1 as taps, dt<0.1 as moving touches.
    perf_df['moving'] = 1
    perf_df.set_value(perf_df[perf_df.dt > 0.1].index, 'moving', 0)
    perf_df = perf_df.set_index(['time'])
    return perf_df[['x', 'y', 'z', 'moving']]

def constrain_perf_df(perf_df):
    perf_df.x = perf_df.x.clip(lower=0.0,upper=1.0)
    perf_df.y = perf_df.y.clip(lower=0.0,upper=1.0)
    perf_df.z = perf_df.z.clip(lower=0.0,upper=1.0)
    return(perf_df)


def random_touch():
    """Generate a random tiny performance touch."""
    return np.array([np.random.rand(), np.random.rand(), 0.01])


def constrain_touch(touch):
    """Constrain touch values from the MDRNN"""
    touch[0] = min(max(touch[0], 0.0), 1.0)  # x in [0,1]
    touch[1] = min(max(touch[1], 0.0), 1.0)  # y in [0,1]
    touch[2] = max(touch[2], 0.001)  # dt # define minimum time step
    return touch

def generate_random_tiny_performance(model, n_mixtures, first_touch, time_limit=5.0, steps_limit=1000, temp=1.0):
    """Generates a tiny performance up to 5 seconds in length."""
    time = 0
    steps = 0
    previous_touch = first_touch
    performance = [previous_touch.reshape((3,))]
    while (steps < steps_limit and time < time_limit):
        params = model.predict(previous_touch.reshape(1,1,3))
        previous_touch = mdn.sample_from_output(params[0], 3, n_mixtures, temp=temp)
        output_touch = previous_touch.reshape(3,)
        output_touch = constrain_touch(output_touch)
        performance.append(output_touch.reshape((3,)))
        steps += 1
        time += output_touch[2]
    return np.array(performance)


def condition_and_generate(model, perf, n_mixtures, time_limit=5.0, steps_limit=1000, temp=1.0):
    """Conditions the network on an existing tiny performance, then generates a new one."""
    time = 0
    steps = 0
    # condition
    for touch in perf:
        params = model.predict(touch.reshape(1,1,3))
        previous_touch = mdn.sample_from_output(params[0], 3, n_mixtures, temp=temp)
    output = [previous_touch.reshape((3,))]
    while (steps < steps_limit and time < time_limit):
        params = model.predict(previous_touch.reshape(1,1,3))
        previous_touch = mdn.sample_from_output(params[0], 3, n_mixtures, temp=temp)
        output_touch = previous_touch.reshape(3,)
        output_touch = constrain_touch(output_touch)
        output.append(output_touch.reshape((3,)))
        steps += 1
        time += output_touch[2]
    net_output = np.array(output)
    return net_output

def divide_performance_into_swipes(perf_df):
    """Divides a performance into a sequence of swipe dataframes for plotting."""
    touch_starts = perf_df[perf_df.moving == 0].index
    performance_swipes = []
    remainder = perf_df
    for att in touch_starts:
        swipe = remainder.iloc[remainder.index < att]
        performance_swipes.append(swipe)
        remainder = remainder.iloc[remainder.index >= att]
    performance_swipes.append(remainder)
    return performance_swipes

def plot_2D(perf_df, name="foo", saving=False):
    """Plot a 2D representation of a performance 2D"""
    swipes = divide_performance_into_swipes(perf_df)
    plt.figure(figsize=(8, 8))
    for swipe in swipes:
        p = plt.plot(swipe.x, swipe.y, 'o-')
        plt.setp(p, color=gen_colour, linewidth=5.0)
    plt.ylim(1.0,0)
    plt.xlim(0,1.0)
    plt.xticks([])
    plt.yticks([])
    if saving:
        plt.savefig(name+".png", bbox_inches='tight')
        plt.close()
    else:
        plt.show()
        
def plot_double_2d(perf1, perf2, name="foo", saving=False):
    """Plot two performances in 2D"""
    plt.figure(figsize=(8, 8))
    swipes = divide_performance_into_swipes(perf1)
    for swipe in swipes:
        p = plt.plot(swipe.x, swipe.y, 'o-')
        plt.setp(p, color=input_colour, linewidth=5.0)
    swipes = divide_performance_into_swipes(perf2)
    for swipe in swipes:
        p = plt.plot(swipe.x, swipe.y, 'o-')
        plt.setp(p, color=gen_colour, linewidth=5.0)
    plt.ylim(1.0,0)
    plt.xlim(0,1.0)
    plt.xticks([])
    plt.yticks([])
    if saving:
        plt.savefig(name+".png", bbox_inches='tight')
        plt.close()
    else:
        plt.show()

In [ ]:
plot_2D(perf_array_to_df(dataset[2000:2050]))

In [ ]:
# Sample a performance

decoder.reset_states()

#n = np.array([np.random.rand(), np.random.rand(), np.random.rand(), 0.02])
n = np.array([0.0, 0.0, 0.0, 0.02])

print(n.shape)

#print(next)
points = []
points.append(n)

for i in range(30):
    params_out = decoder.predict(np.array([[n]]))[0][0]
    #print(params_out.shape)
    n_out = sample_from_output(params_out, OUTPUT_DIMENSION, NUMBER_MIXTURES, temp=1.0, sig_temp=0.01)
    #print(n_out[0].shape)
    points.append(n_out[0])
    n_in = n_out

output = np.array(points)
#print(output)
print(output.shape)

p_df = perf_array_to_df(output)
p_df = constrain_perf_df(p_df)
print(p_df)
plot_2D(p_df)

In [ ]:
def split_mixture_params(params, output_dim, num_mixes):
    """Splits up an array of mixture parameters into mus, sigmas, and pis
    depending on the number of mixtures and output dimension."""
    mus = params[:num_mixes*output_dim]
    sigs = params[num_mixes*output_dim:2*num_mixes*output_dim]
    pi_logits = params[-num_mixes:]
    return mus, sigs, pi_logits


def softmax(w, t=1.0):
    """Softmax function for a list or numpy array of logits. Also adjusts temperature."""
    e = np.array(w) / t  # adjust temperature
    e -= e.max()  # subtract max to protect from exploding exp values.
    e = np.exp(e)
    dist = e / np.sum(e)
    return dist


def sample_from_categorical(dist):
    """Samples from a categorical model PDF."""
    r = np.random.rand(1)  # uniform random number in [0,1]
    accumulate = 0
    for i in range(0, dist.size):
        accumulate += dist[i]
        if accumulate.any() >= r[0]:
            return i
    tf.logging.info('Error sampling mixture model.')
    return -1


def sample_from_output(params, output_dim, num_mixes, temp=1.0, sig_temp=1.0):
    """Sample from an MDN output with temperature adjustment."""
    mus = params[:num_mixes*output_dim]
    sigs = params[num_mixes*output_dim:2*num_mixes*output_dim]
    pis = softmax(params[-num_mixes:], t=temp)
    m = sample_from_categorical(pis)
    # Alternative way to sample from categorical:
    # m = np.random.choice(range(len(pis)), p=pis)
    mus_vector = mus[m*output_dim:(m+1)*output_dim]
    sig_vector = sigs[m*output_dim:(m+1)*output_dim] * sig_temp  # adjust for temperature
    cov_matrix = np.identity(output_dim) * sig_vector
    sample = np.random.multivariate_normal(mus_vector, cov_matrix, 1)
    return sample

In [ ]:
    r = np.random.rand(1)  # uniform random number in [0,1]

0 >= r[0]